<a id='importers'></a>
<p style="font-weight:bold;"> <span style="font-size: 36px"> Importer Methods </span> </p>


In [0]:
#!import "ImportScopeCalculation"

# Parsing Storage

The Parsing storage collects data required for the import of a file. 
Such storage is then passed to parsing format definitions to map the content of the file into variables.

In [0]:
public class ParsingStorage
{
    private readonly IDataSource dataSource;
    private readonly IWorkspace workspace;
    private readonly ImportArgs args;
    
    //Hierarchy Cache
    public Systemorph.Vertex.Hierarchies.IHierarchicalDimensionCache HierarchyCache;
    
    public ReportingNode ReportingNode { get; protected set; }
    
    public Dictionary<string, DataNodeData> DataNodeDataBySystemName;
    
    // Dimensions
    public Dictionary<string, EstimateType> EstimateType;
    public Dictionary<string, AmountType> AmountType; 
    public HashSet<AocStep> AocTypeMap;
    private HashSet<string> estimateTypes;
    private HashSet<string> amountTypes;
    private Dictionary<string, HashSet<string>> amountTypesByEstimateType => GetAmountTypesByEstimateType(HierarchyCache);
    public HashSet<string> TechnicalMarginEstimateTypes => GetTechnicalMarginEstimateType(); 
    public Dictionary<Type, Dictionary<string, string>> DimensionsWithExternalId;
    
    // Partitions
    public PartitionByReportingNode TargetPartitionByReportingNode;
    public PartitionByReportingNodeAndPeriod TargetPartitionByReportingNodeAndPeriod;
    
    //Constructor
    public ParsingStorage(ImportArgs args, IDataSource dataSource, IWorkspace workspace)
    {
        this.args = args;
        this.dataSource = dataSource;
        this.workspace = workspace;
    }
    
    // Initialize
    public async Task InitializeAsync()
    {
        //Partition Workspace and DataSource
        TargetPartitionByReportingNode = (await workspace.Query<PartitionByReportingNode>().Where(p => p.ReportingNode == args.ReportingNode).ToArrayAsync()).SingleOrDefault(); 
        
        if(TargetPartitionByReportingNode == null) 
        { ApplicationMessage.Log(Error.ParsedPartitionNotFound, args.ReportingNode); return; } 
        
        await workspace.Partition.SetAsync<PartitionByReportingNode>(TargetPartitionByReportingNode.Id);
        await dataSource.Partition.SetAsync<PartitionByReportingNode>(TargetPartitionByReportingNode.Id);
        
        if(args.Year != default(int) && args.Month != default(int))
        {
            TargetPartitionByReportingNodeAndPeriod = (await workspace.Query<PartitionByReportingNodeAndPeriod>()
                                                                      .Where(p => p.ReportingNode == args.ReportingNode &&
                                                                             p.Year == args.Year &&
                                                                             p.Month == args.Month &&
                                                                             p.Scenario == args.Scenario).ToArrayAsync()).SingleOrDefault();
            
            if(TargetPartitionByReportingNodeAndPeriod == null) 
            { ApplicationMessage.Log(Error.ParsedPartitionNotFound, args.ReportingNode, args.Year.ToString(), args.Month.ToString(), args.Scenario); return; } 
        
            await workspace.Partition.SetAsync<PartitionByReportingNodeAndPeriod>(TargetPartitionByReportingNodeAndPeriod.Id);
            await dataSource.Partition.SetAsync<PartitionByReportingNodeAndPeriod>(TargetPartitionByReportingNodeAndPeriod.Id);
            
            //Clean up the workspace
            await workspace.DeleteAsync<RawVariable>( await workspace.Query<RawVariable>().ToArrayAsync() );
            await workspace.DeleteAsync<IfrsVariable>( await workspace.Query<IfrsVariable>().ToArrayAsync() );
        }
        
        var reportingNodes = (await dataSource.Query<ReportingNode>().Where(x => x.SystemName == args.ReportingNode).ToArrayAsync());
        if(!reportingNodes.Any()) { ApplicationMessage.Log(Error.ReportingNodeNotFound, args.ReportingNode); return; }
        ReportingNode = reportingNodes.First();

        var aocConfigurationByAocStep = await dataSource.LoadAocStepConfigurationAsync(args.Year, args.Month);
        AocTypeMap = args.ImportFormat switch {
                ImportFormats.Cashflow => aocConfigurationByAocStep.Where(x => x.InputSource.Contains(InputSource.Cashflow) &&
                                                                                !new DataType[]{DataType.Calculated, DataType.CalculatedTelescopic}.Contains(x.DataType) )
                                                                   .GroupBy(x => new AocStep(x.AocType, x.Novelty), (k,v) => k).ToHashSet(),
                ImportFormats.Actual => aocConfigurationByAocStep.Where(x => x.InputSource.Contains(InputSource.Actual) &&
                                                                                !new DataType[]{DataType.Calculated, DataType.CalculatedTelescopic}.Contains(x.DataType) && 
                                                                                 new AocStep(x.AocType, x.Novelty) != new AocStep(AocTypes.BOP, Novelties.I))
                                                                 .GroupBy(x => new AocStep(x.AocType, x.Novelty), (k,v) => k).ToHashSet(),
                ImportFormats.Opening => aocConfigurationByAocStep.Where(x => x.InputSource.Contains(InputSource.Opening) && x.DataType == DataType.Optional).GroupBy(x => new AocStep(x.AocType, x.Novelty), (k,v) => k).ToHashSet(),
                ImportFormats.SimpleValue => aocConfigurationByAocStep.GroupBy(x => new AocStep(x.AocType, x.Novelty), (k,v) => k).Concat((await dataSource.Query<PnlVariableType>().ToArrayAsync())
                                                                                   .Select(vt => new AocStep(vt.SystemName,null))).ToHashSet(),
                _ => Enumerable.Empty<AocStep>().ToHashSet(),
        };
        
        //DataNodes
        DataNodeDataBySystemName = args.ImportFormat == ImportFormats.Opening 
                                    ? (await LoadDataNodesAsync(dataSource, args)).Where(kvp => kvp.Value.Year == args.Year).ToDictionary(kvp => kvp.Key, kvp => kvp.Value)
                                    : await LoadDataNodesAsync(dataSource, args);
        // Dimensions
        EstimateType = (await dataSource.Query<EstimateType>().ToArrayAsync()).ToDictionary(x => x.SystemName);
        AmountType = (await dataSource.Query<AmountType>().Where(x =>!(x is DeferrableAmountType)).ToArrayAsync()).ToDictionary(x => x.SystemName);
        amountTypes = (await dataSource.Query<AmountType>().ToArrayAsync()).Select(at => at.SystemName).ToHashSet();
        estimateTypes = args.ImportFormat switch {
                ImportFormats.SimpleValue => (await dataSource.Query<EstimateType>().ToArrayAsync()).Select(et => et.SystemName).ToHashSet(),
                ImportFormats.Opening => (await dataSource.Query<EstimateType>().Where(et => et.StructureType == StructureType.AoC).ToArrayAsync())
                                                                                .Where(et => et.InputSource.Contains(InputSource.Opening)) //This Contains overload cannot be used in DB
                                                                                .Select(et => et.SystemName).ToHashSet(),
                _ => Enumerable.Empty<string>().ToHashSet(),
        };
        
        
        // DimensionsWithExternalId
        DimensionsWithExternalId = new Dictionary<Type, Dictionary<string, string>>()
        {
            { typeof(AmountType), await GetDimensionWithExternalIdDictionaryAsync<AmountType>() },
            { typeof(EstimateType), await GetDimensionWithExternalIdDictionaryAsync<EstimateType>() }
        };
        
        //Hierarchy Cache
        HierarchyCache = workspace.ToHierarchicalDimensionCache();
        await HierarchyCache.InitializeAsync<AmountType>();
    }
    
    public async Task<Dictionary<string, string>> GetDimensionWithExternalIdDictionaryAsync<T> () where T : KeyedOrderedDimension
    {
        var dict = new Dictionary<string, string>();
        var items = await dataSource.Query<T>().ToArrayAsync();
        foreach (var item in items) {
            dict.TryAdd(item.SystemName, item.SystemName);
            if(typeof(T).IsAssignableTo(typeof(KeyedOrderedDimensionWithExternalId))) {
                var externalIds = (string[])(typeof(T).GetProperty(nameof(KeyedOrderedDimensionWithExternalId.ExternalId)).GetValue(item));
                if(externalIds == null) continue;
                foreach (var extId in externalIds) 
                    dict.TryAdd(extId, item.SystemName);
            }
        }
        return dict;
    }
    
    // Getters
    public bool IsDataNodeReinsurance(string goc) => DataNodeDataBySystemName[goc].IsReinsurance;
    public bool IsValidDataNode(string goc) => DataNodeDataBySystemName.ContainsKey(goc);
    
    // Validations
    public string ValidateEstimateType(string et, string goc) {
        var allowedEstimateTypes = estimateTypes;
        if (DataNodeDataBySystemName.TryGetValue(goc, out var dataNodeData) && dataNodeData.LiabilityType == LiabilityTypes.LIC)
            estimateTypes.ExceptWith(TechnicalMarginEstimateTypes);
        if(!allowedEstimateTypes.Contains(et))
            ApplicationMessage.Log(Error.EstimateTypeNotFound, et);
        return et;
    }
    
    public string ValidateAmountType(string at) {
        if (at != null && !amountTypes.Contains(at))
           ApplicationMessage.Log(Error.AmountTypeNotFound, at);
        return at;
    }
    
    public AocStep ValidateAocStep(AocStep aoc) {
        if (!AocTypeMap.Contains(aoc))
            ApplicationMessage.Log(Error.AocTypeMapNotFound, aoc.AocType, aoc.Novelty);
        return aoc;
    }
    
    public string ValidateDataNode(string goc) {
        if (!DataNodeDataBySystemName.ContainsKey(goc))
            ApplicationMessage.Log(Error.InvalidDataNode, goc);
        return goc;
    }
    
    public void ValidateEstimateTypeAndAmountType(string estimateType, string amountType){
        if (amountTypesByEstimateType.TryGetValue(estimateType, out var ats) && ats.Any() && !ats.Contains(amountType))
            ApplicationMessage.Log(Error.InvalidAmountTypeEstimateType, estimateType, amountType);
    }
}

# Basics

Basic methods to delete variable and update variables to the data source are defined.

## Clean the Database

In [0]:
async public Task CleanDatabaseAsync<T> (Expression<Func<T, bool>> filter = null) where T : class
{
    var loadData = await DataSource.Query<T>().Where(filter?? (Expression<Func<T, bool>>)(x => true)).ToListAsync();
    await DataSource.DeleteAsync(loadData);
}

## Update the Database

In [0]:
public async Task CommitToDatabase<TData, TPartition> (IWorkspace workspace, Guid partitionId = new Guid(), bool snapshot = true, Expression<Func<TData, bool>> filter = null) 
where TData : class, IPartitioned
where TPartition : IfrsPartition
{
    if(partitionId != new Guid()) {
        await workspace.Partition.SetAsync<TPartition>(partitionId);
        await DataSource.Partition.SetAsync<TPartition>(partitionId);
    }
    if(snapshot) await CleanDatabaseAsync<TData>(filter);
    await DataSource.UpdateAsync<TData>( await workspace.Query<TData>().ToArrayAsync() );
    await DataSource.CommitAsync();
}

# Import helpers

## Parse Main Tab and return Args

The main table of our custom import formats contains the information which are required to identify the data partition. These information are temporarily stored in [Args](../DataModel/DataStructure#args) and used in the next methods.

In [0]:
public async Task<ImportArgs> GetArgsFromMainAsync<IPartition>(IDataSet dataSet)
{
    var mainTab = dataSet.Tables[Main];
    if(mainTab == null) ApplicationMessage.Log(Error.NoMainTab);
    if(mainTab.Rows.Count() == 0) ApplicationMessage.Log(Error.IncompleteMainTab);
    if(mainTab.Columns.FirstOrDefault(x => x.ColumnName == nameof(ReportingNode)) == null) ApplicationMessage.Log(Error.ReportingNodeInMainNotFound);
    if(ApplicationMessage.HasErrors()) return null;
    
    var main = mainTab.Rows.First();
    var reportingNode = (string)main[nameof(ReportingNode)];
    var scenario = mainTab.Columns.Where(x => x.ColumnName == nameof(Scenario)).Count()>0? (string)main[nameof(Scenario)] : default(string);
   
    if(ApplicationMessage.HasErrors()) return null;

    ImportArgs args;
    switch(typeof(IPartition).Name) {
        case nameof(PartitionByReportingNode) : {
            args = new ImportArgs( reportingNode,
                                   default(int),
                                   default(int),
                                   default(Periodicity),
                                   scenario,
                                   default(string));
            await DataSource.UpdateAsync<PartitionByReportingNode>( new[] { new PartitionByReportingNode { 
                                   Id = (Guid)(await DataSource.Partition.GetKeyForInstanceAsync<PartitionByReportingNode>(args)),
                                   ReportingNode = reportingNode } } );
            break;
        }
        case nameof(PartitionByReportingNodeAndPeriod) : {
            if(mainTab.Columns.Where(x => x.ColumnName == nameof(PartitionByReportingNodeAndPeriod.Year)).Count()!=1) ApplicationMessage.Log(Error.YearInMainNotFound);
            if(mainTab.Columns.Where(x => x.ColumnName == nameof(PartitionByReportingNodeAndPeriod.Month)).Count()!=1) ApplicationMessage.Log(Error.MonthInMainNotFound);
            if(ApplicationMessage.HasErrors()) return null;
            args = new ImportArgs( reportingNode, 
                                   (int)Convert.ChangeType(main[nameof(PartitionByReportingNodeAndPeriod.Year)], typeof(int)),
                                   (int)Convert.ChangeType(main[nameof(PartitionByReportingNodeAndPeriod.Month)], typeof(int)),
                                   default(Periodicity),
                                   scenario,
                                   default(string) );
            await DataSource.UpdateAsync<PartitionByReportingNodeAndPeriod>( new[]{ new PartitionByReportingNodeAndPeriod { 
                                   Id = (Guid)(await DataSource.Partition.GetKeyForInstanceAsync<PartitionByReportingNodeAndPeriod>(args)),
                                   Year = args.Year,
                                   Month = args.Month,
                                   ReportingNode = reportingNode, 
                                   Scenario = scenario } } );
            break;
        }
        default : {
            ApplicationMessage.Log(Error.PartitionTypeNotFound, typeof(IPartition).Name); 
            return null;
        }
    }
    
    await DataSource.CommitAsync();
    return args;
}

## Data Node Factory

In [0]:
public async Task DataNodeFactoryAsync(IDataSet dataSet, string tableName, ImportArgs args)
{
    var partition = (await DataSource.Query<PartitionByReportingNode>().Where(p => p.ReportingNode == args.ReportingNode).ToArrayAsync()).FirstOrDefault();
    if(partition == null) { ApplicationMessage.Log(Error.ParsedPartitionNotFound); return; }
    
    var table = dataSet.Tables[tableName];
    
    var dataNodesImported = table.Rows.Select(x => x.Field<string>(nameof(RawVariable.DataNode))).ToHashSet();
    var dataNodesDefined = await DataSource.Query<GroupOfContract>().Where(x => dataNodesImported.Contains(x.SystemName)).ToArrayAsync();
    var dataNodeStatesDefined = await DataSource.Query<DataNodeState>().Select(x => x.DataNode).ToArrayAsync();
    var dataNodeParametersDefined = await DataSource.Query<SingleDataNodeParameter>().Select(x => x.DataNode).ToArrayAsync();  
    var dataNodeStatesUndefined = dataNodesImported.Where(x => x != null && !dataNodeStatesDefined.Contains(x)).ToHashSet();
    var dataNodeSingleParametersUndefined = dataNodesImported.Where(x => x != null &&
                                                                    !dataNodeParametersDefined.Contains(x) && 
                                                                    dataNodesDefined.SingleOrDefault(y => y.SystemName == x) is GroupOfInsuranceContract).ToHashSet();
    await DataSource.UpdateAsync( dataNodeStatesUndefined.Select(x => 
        new DataNodeState {DataNode = x, Year = args.Year, Month = DefaultDataNodeActivationMonth, State = State.Active, Partition = partition.Id}).ToArray() );
    await DataSource.UpdateAsync( dataNodeSingleParametersUndefined.Select(x => 
        new SingleDataNodeParameter {DataNode = x, Year = args.Year, Month = DefaultDataNodeActivationMonth, PremiumAllocation = DefaultPremiumExperienceAdjustmentFactor, Partition = partition.Id}).ToArray() );
    await DataSource.CommitAsync();
}

## Validation for Active Data Node States

In [0]:
public async Task ValidateForDataNodeStateActiveAsync<T>(IWorkspace workspace, Dictionary<string, DataNodeData> dataNodes) where T : BaseDataRecord
{   
    foreach(var item in (await workspace.Query<T>().ToArrayAsync()).GroupBy(x => x.DataNode))
        if(!dataNodes.ContainsKey(item.First().DataNode))
            ApplicationMessage.Log(Error.InactiveDataNodeState, item.First().DataNode);
}


## Get Target Args

In [0]:
public async Task<ImportArgs[]> GetTargetArgsAsync(ImportArgs mainArgs)
{
    var scenarioArgs = await DataSource.Query<PartitionByReportingNodeAndPeriod>() 
                               .Where(x => x.ReportingNode == mainArgs.ReportingNode && x.Year == mainArgs.Year && x.Month == mainArgs.Month && x.Scenario != null)
                               .Select(x => new ImportArgs(x.ReportingNode, x.Year, x.Month, default(Periodicity), x.Scenario, ImportFormats.Cashflow)).ToArrayAsync();
    var isMultipleArg = mainArgs.Scenario == null && scenarioArgs.Any();
    var targetArgs = isMultipleArg
                     ? mainArgs.RepeatOnce().Concat(scenarioArgs)
                     : mainArgs.RepeatOnce();

    if (isMultipleArg) ApplicationMessage.Log(Warning.ScenarioReCalculations, String.Join(", ", scenarioArgs.Select(x => x.Scenario)));
    return targetArgs.ToArray();
}   

## Calculate and Upload IFRS Variables

In [0]:
public async Task<ActivityLog> CalculateAndUploadVariablesAsync(ImportArgs args, IWorkspace workspace, bool saveRawVariables)
{
    Activity.Start();
    var storage = new ImportStorage(args, DataSource, workspace);
    await storage.InitializeAsync();
    if(Activity.HasErrors()) return Activity.Finish();
   
    var universe = Scopes.ForStorage(storage).ToScope<IModel>();
    var identities = universe.GetScopes<GetIdentities>(storage.DataNodesByImportScope[ImportScope.Primary]).SelectMany(s => s.Identities);
    var ivs = universe.GetScopes<ComputeAllScopes>(identities).SelectMany(x => x.CalculatedIfrsVariables);
    if(Activity.HasErrors()) return Activity.Finish();

    if(storage.DefaultPartition != storage.TargetPartition){
        var bestEstimateIvs = await DataSource.LoadPartitionedDataAsync<IfrsVariable,PartitionByReportingNodeAndPeriod>(storage.DefaultPartition, storage.TargetPartition);
        ivs = ivs.Where(iv => Math.Abs(iv.Value) >= Precision).ToArray().Concat(
              ivs.Where(iv => Math.Abs(iv.Value) < Precision).Intersect(bestEstimateIvs, EqualityComparer<IfrsVariable>.Instance).Select(iv => iv with {Value = 0.0}).ToArray());
    }


    await workspace.UpdateAsync<IfrsVariable>(ivs.Where(x => storage.DefaultPartition != storage.TargetPartition || Math.Abs(x.Value) >= Precision));
    await CommitToDatabase<IfrsVariable,PartitionByReportingNodeAndPeriod>(workspace, storage.TargetPartition, snapshot : true, 
                                                                           filter : x => storage.EstimateTypesByImportFormat[args.ImportFormat].Contains(x.EstimateType) 
                                                                                      && storage.DataNodesByImportScope[ImportScope.Primary].Contains(x.DataNode));
    if(saveRawVariables) {
        if(args.Scenario == null) await workspace.DeleteAsync(await workspace.Query<RawVariable>().Where(rv => rv.Values.Sum(x => Math.Abs(x)) < Precision).ToArrayAsync());
        await CommitToDatabase<RawVariable, PartitionByReportingNodeAndPeriod>(workspace, storage.TargetPartition, snapshot : true, 
                                                                               filter : x => storage.DataNodesByImportScope[ImportScope.Primary].Contains(x.DataNode));
    }
    return Activity.Finish();
}

# Analysis of Change Configuration

The [Analysis of Change configuration](../DataModel/DataStructure#aoc-step-configuration) is parsed from the input file and complemented with defaults to allow for an easy insertion of new AoC Steps.

After having checked that the [AocTypes](../DataModel/DataStructure#aoc-variable-type) loaded in the target DataSource are including all the compulsory ones, default configurations are generated on the basis of the AocTypes ordering. 

The following categories have been identified based on the *Order* of the novel AoC Step:

**Category**   $$\hspace{2.8cm}$$ **Default added with same configuration of**
|    |              |
|-----------------------|-------------------------------------------------------|
| Order < RCU $$\phantom{.......................}$$ |  MC with Novelty I                                   |
| RCU < Order < CF      |  RCU with Novelty I                                   |
| IA  < Order < YCU     |  AU with both Novelty I and N                         |
| CRU < Order < WO      |  EV with Novelty I and N                              |
| WO  < Order < CL      |  WO with Novelty C (only for Import Source = Actual)  |

The new AoC Configurations are created with the same order of the AoC Types. 

In [0]:
Import.DefineFormat(ImportFormats.AocConfiguration, async (options, dataSet) => {
    Activity.Start();
    var workspace = Workspace.CreateNew();
    workspace.InitializeFrom(options.TargetDataSource);

    var aocTypes = await options.TargetDataSource.Query<AocType>().OrderBy(x => x.Order).ToArrayAsync();
    var aocTypesCompulsory = typeof(AocTypes).GetFields().Select(x => (string)x.Name);
    if(aocTypesCompulsory.Where(x => !aocTypes.Select(x => x.SystemName).Contains(x)).Any()) {
        ApplicationMessage.Log(Error.AocTypeCompulsoryNotFound);
        return Activity.Finish();
    }
    
    var logConfig = await Import.FromDataSet(dataSet).WithType<AocConfiguration>().WithTarget(workspace).ExecuteAsync();
    if(logConfig.Errors.Any()) return Activity.Finish().Merge(logConfig); 

    var orderByName = aocTypes.ToDictionary(x => x.SystemName, x => x.Order);
    var aocConfigs = (await workspace.Query<AocConfiguration>().ToArrayAsync())
                                     .GroupBy(x => (x.AocType, x.Novelty))
                                     .Select(y => y.OrderByDescending(x => x.Year).ThenByDescending(x => x.Month).FirstOrDefault())
                                     .ToDictionary(x => (x.AocType, x.Novelty));
    var aocOrder = aocConfigs.ToDictionary(x => x.Key, x => x.Value.Order);
    var newAocTypes = orderByName.Keys.Where(x => !aocConfigs.Keys.Contains((x, Novelties.I)) && 
                                                  !aocConfigs.Keys.Contains((x, Novelties.N)) && 
                                                  !aocConfigs.Keys.Contains((x, Novelties.C)) && 
                                                  !aocTypes.Any(y => y.Parent == x)           &&
                                                  !aocTypesCompulsory.Contains(x)).ToArray();

    foreach(var newAocType in newAocTypes) {
        if(orderByName[newAocType] < orderByName[AocTypes.RCU])
        {
            var step = (AocTypes.MC, Novelties.I);
            await workspace.UpdateAsync( aocConfigs[step] with { AocType = newAocType, DataType = DataType.Optional, Order = ++aocOrder[step] }); 
        }
        else if(orderByName[newAocType] > orderByName[AocTypes.RCU] && orderByName[newAocType] < orderByName[AocTypes.CF])  
        {
            var step = (AocTypes.RCU, Novelties.I);
            await workspace.UpdateAsync( aocConfigs[step] with { AocType = newAocType, DataType = DataType.Optional, Order = ++aocOrder[step] });
        }
        else if(orderByName[newAocType] > orderByName[AocTypes.IA] && orderByName[newAocType] < orderByName[AocTypes.YCU])  
        {
            foreach (var novelty in new[]{Novelties.I, Novelties.N}) {
                var step = (AocTypes.AU, novelty);
                var order = orderByName[newAocType] < orderByName[AocTypes.AU]? ++aocOrder[(AocTypes.IA, novelty)] : ++aocOrder[(AocTypes.AU, novelty)];
                await workspace.UpdateAsync( aocConfigs[step] with { AocType = newAocType, DataType = DataType.Optional, Order = order } );
            }
        }
        else if(orderByName[newAocType] > orderByName[AocTypes.CRU] && orderByName[newAocType] < orderByName[AocTypes.WO])
        {
            var stepI = (AocTypes.EV, Novelties.I);
            var orderI = orderByName[newAocType] < orderByName[AocTypes.EV]? ++aocOrder[(AocTypes.CRU, Novelties.I)] : ++aocOrder[(AocTypes.EV, Novelties.I)];
            await workspace.UpdateAsync( aocConfigs[stepI] with { AocType = newAocType, DataType = DataType.Optional, Order = orderI } );

            var stepN = (AocTypes.EV, Novelties.N);
            var orderN = orderByName[newAocType] < orderByName[AocTypes.EV]? ++aocOrder[(AocTypes.AU, Novelties.N)] : ++aocOrder[(AocTypes.EV, Novelties.N)];
            await workspace.UpdateAsync( aocConfigs[stepN] with { AocType = newAocType, DataType = DataType.Optional, Order = orderN } );
        }
        else if(orderByName[newAocType] > orderByName[AocTypes.WO] && orderByName[newAocType] < orderByName[AocTypes.CL])
        {
            var step = (AocTypes.WO, Novelties.C);
            await workspace.UpdateAsync( aocConfigs[step] with { AocType = newAocType, DataType = DataType.Optional, Order = ++aocOrder[step] } );
        }
        else
            ApplicationMessage.Log(Error.AocTypePositionNotSupported);
    };

    var aocConfigsFinal = await workspace.Query<AocConfiguration>().ToArrayAsync();
    if(aocConfigsFinal.GroupBy(x => x.Order).Any(x => x.Count() > 1))
        ApplicationMessage.Log(Error.AocConfigurationOrderNotUnique);

    await workspace.CommitToTargetAsync(options.TargetDataSource);
    return Activity.Finish().Merge(logConfig); 
});

# Data Nodes



[Portfolios](../DataModel/DataStructure#portfolios) and [Group of Contracts](../DataModel/DataStructure#group-of-contracts) are imported in the same file.  

In [0]:
public async Task<ActivityLog> UploadDataNodesToWorkspaceAsync(IDataSet dataSet, IWorkspace workspace)
{
    workspace.Reset(x => x.ResetInitializationRules().ResetCurrentPartitions());
    workspace.Initialize(x => x.FromSource(DataSource)
                               .DisableInitialization<RawVariable>()
                               .DisableInitialization<IfrsVariable>()
                               .DisableInitialization<DataNodeState>()
                               .DisableInitialization<DataNodeParameter>());
    
    Activity.Start();
    var args = await GetArgsFromMainAsync<PartitionByReportingNode>(dataSet);
    if(Activity.HasErrors()) return Activity.Finish();
    
    var storage = new ParsingStorage(args, DataSource, workspace);
    await storage.InitializeAsync();
    if(Activity.HasErrors()) return Activity.Finish();
       
    var importLogPortfolios = await Import.FromDataSet(dataSet)
        .WithType<InsurancePortfolio>((dataset, datarow) => 
            new InsurancePortfolio { SystemName = datarow.Field<string>(nameof(DataNode.SystemName)),
                                     DisplayName = datarow.Field<string>(nameof(DataNode.DisplayName)),
                                     Partition = storage.TargetPartitionByReportingNode.Id,
                                     ContractualCurrency = datarow.Field<string>(nameof(DataNode.ContractualCurrency)),
                                     FunctionalCurrency = storage.ReportingNode.Currency,
                                     LineOfBusiness = datarow.Field<string>(nameof(DataNode.LineOfBusiness)),
                                     ValuationApproach = datarow.Field<string>(nameof(DataNode.ValuationApproach)),
                                     OciType = datarow.Field<string>(nameof(DataNode.OciType)) })
        .WithType<ReinsurancePortfolio>((dataset, datarow) =>
            new ReinsurancePortfolio {  SystemName = datarow.Field<string>(nameof(DataNode.SystemName)),
                                        DisplayName = datarow.Field<string>(nameof(DataNode.DisplayName)),
                                        Partition = storage.TargetPartitionByReportingNode.Id,
                                        ContractualCurrency = datarow.Field<string>(nameof(DataNode.ContractualCurrency)),
                                        FunctionalCurrency = storage.ReportingNode.Currency,
                                        LineOfBusiness = datarow.Field<string>(nameof(DataNode.LineOfBusiness)),
                                        ValuationApproach = datarow.Field<string>(nameof(DataNode.ValuationApproach)),
                                        OciType = datarow.Field<string>(nameof(DataNode.OciType)) })
        .WithTarget(workspace)
        .ExecuteAsync();
    
    var portfolios = await workspace.Query<Portfolio>().ToDictionaryAsync(x => x.SystemName);
    var yieldCurveColumnGroupOfInsuranceContract = dataSet.Tables[nameof(GroupOfInsuranceContract)].Columns.Any(x => x.ColumnName == nameof(GroupOfInsuranceContract.YieldCurveName));
    var yieldCurveColumnGroupOfReinsuranceContract = dataSet.Tables[nameof(GroupOfReinsuranceContract)].Columns.Any(x => x.ColumnName == nameof(GroupOfReinsuranceContract.YieldCurveName));

    var importLogGroupOfContracts = await Import.FromDataSet(dataSet)
        .WithType<GroupOfInsuranceContract>((dataset, datarow) => {
            var gicSystemName = datarow.Field<string>(nameof(DataNode.SystemName));
            var pf = datarow.Field<string>(nameof(InsurancePortfolio));
            if(!portfolios.TryGetValue(pf, out var portfolioData)) {
                ApplicationMessage.Log(Error.PortfolioGicNotFound, pf, gicSystemName);
                return null;
            }
            var gic = new GroupOfInsuranceContract { SystemName = gicSystemName,
                                                     DisplayName = datarow.Field<string>(nameof(DataNode.DisplayName)),
                                                     Partition = storage.TargetPartitionByReportingNode.Id,
                                                     ContractualCurrency = portfolioData.ContractualCurrency,
                                                     FunctionalCurrency = portfolioData.FunctionalCurrency,
                                                     LineOfBusiness = portfolioData.LineOfBusiness,
                                                     ValuationApproach = portfolioData.ValuationApproach,
                                                     OciType = portfolioData.OciType,
                                                     AnnualCohort =  Convert.ToInt32(datarow.Field<object>(nameof(GroupOfContract.AnnualCohort))),
                                                     LiabilityType = datarow.Field<string>(nameof(GroupOfContract.LiabilityType)),
                                                     Profitability = datarow.Field<string>(nameof(GroupOfContract.Profitability)),
                                                     Portfolio = pf,
                                                     YieldCurveName = yieldCurveColumnGroupOfInsuranceContract
                                                                    ? datarow.Field<string>(nameof(GroupOfContract.YieldCurveName)) 
                                                                    : (string)null };
            return ExtendGroupOfContract(gic, datarow);
        })
        .WithType<GroupOfReinsuranceContract>((dataset, datarow) => {
            var gricSystemName = datarow.Field<string>(nameof(DataNode.SystemName));
            var pf = datarow.Field<string>(nameof(ReinsurancePortfolio));
            if(!portfolios.TryGetValue(pf, out var portfolioData)) {
                ApplicationMessage.Log(Error.PortfolioGicNotFound, pf, gricSystemName);
                return null;
            }
            var gric = new GroupOfReinsuranceContract { SystemName = gricSystemName,
                                                        DisplayName = datarow.Field<string>(nameof(DataNode.DisplayName)),
                                                        Partition = storage.TargetPartitionByReportingNode.Id,
                                                        ContractualCurrency = portfolioData.ContractualCurrency,
                                                        FunctionalCurrency = portfolioData.FunctionalCurrency,
                                                        LineOfBusiness = portfolioData.LineOfBusiness,
                                                        ValuationApproach = portfolioData.ValuationApproach,
                                                        OciType = portfolioData.OciType,
                                                        AnnualCohort = Convert.ToInt32(datarow.Field<object>(nameof(GroupOfContract.AnnualCohort))),
                                                        LiabilityType = datarow.Field<string>(nameof(GroupOfContract.LiabilityType)),
                                                        Profitability = datarow.Field<string>(nameof(GroupOfContract.Profitability)),
                                                        Portfolio = pf,
                                                        Partner = datarow.Field<string>(nameof(GroupOfContract.Partner)),
                                                        YieldCurveName = yieldCurveColumnGroupOfReinsuranceContract
                                                                        ? datarow.Field<string>(nameof(GroupOfContract.YieldCurveName)) 
                                                                        : (string)null };
            return ExtendGroupOfContract(gric, datarow);
        })
        .WithTarget(workspace)
        .ExecuteAsync();
   
    return Activity.Finish().Merge(importLogPortfolios).Merge(importLogGroupOfContracts);
}

In [0]:
Import.DefineFormat(ImportFormats.DataNode, async (options, dataSet) => {
    var workspace = Workspace.CreateNew();
    var log = await UploadDataNodesToWorkspaceAsync(dataSet, workspace);
    var partition = (Guid)workspace.Partition.GetCurrent(nameof(PartitionByReportingNode));
    await CommitToDatabase<InsurancePortfolio,PartitionByReportingNode>(workspace, partition);
    await CommitToDatabase<ReinsurancePortfolio,PartitionByReportingNode>(workspace, partition);
    await CommitToDatabase<GroupOfInsuranceContract,PartitionByReportingNode>(workspace, partition);
    await CommitToDatabase<GroupOfReinsuranceContract,PartitionByReportingNode>(workspace, partition);
    return log;
});

# Data Node State

In [0]:
public async static Task ValidateDataNodeStatesAsync(this IWorkspace workspace, Dictionary<string, DataNodeData> persistentDataNodeByDataNode)
{
    foreach(var importedDataNodeState in await workspace.Query<DataNodeState>().ToArrayAsync())
    {
        if(persistentDataNodeByDataNode.TryGetValue(importedDataNodeState.DataNode, out var currentPersistentDataNode))
        {
            if(importedDataNodeState.State < currentPersistentDataNode.State)
                ApplicationMessage.Log(Error.ChangeDataNodeState, importedDataNodeState.DataNode, 
                                                                  currentPersistentDataNode.State.ToString(), 
                                                                  importedDataNodeState.State.ToString());

            if(importedDataNodeState.State == currentPersistentDataNode.State)
                await workspace.DeleteAsync<DataNodeState>(importedDataNodeState);
        }
    }
}

In [0]:
public async Task<ActivityLog> UploadDataNodeStateToWorkspaceAsync(IDataSet dataSet, IWorkspace workspace)
{
    workspace.Reset(x => x.ResetInitializationRules().ResetCurrentPartitions());
    workspace.Initialize(x => x.FromSource(DataSource)
                               .DisableInitialization<RawVariable>()
                               .DisableInitialization<IfrsVariable>()
                               .DisableInitialization<DataNodeState>());
    Activity.Start();
    var args = await GetArgsFromMainAsync<PartitionByReportingNodeAndPeriod>(dataSet);
    if(Activity.HasErrors()) return Activity.Finish();
    
    var storage = new ParsingStorage(args, DataSource, workspace);
    await storage.InitializeAsync();
    if(Activity.HasErrors()) return Activity.Finish();

    var importLog = await Import.FromDataSet(dataSet).WithType<DataNodeState>(
        (dataset, datarow) => new DataNodeState {
            DataNode = datarow.Field<string>(nameof(DataNodeState.DataNode)),
            State = (State)Enum.Parse(typeof(State), datarow.Field<string>(nameof(DataNodeState.State))),
            Year = args.Year,
            Month = args.Month,
            Partition = storage.TargetPartitionByReportingNode.Id
        }
    ).WithTarget(workspace).ExecuteAsync();

    await workspace.ValidateDataNodeStatesAsync(storage.DataNodeDataBySystemName);
    return Activity.Finish().Merge(importLog);
}

In [0]:
Import.DefineFormat(ImportFormats.DataNodeState, async (options, dataSet) => {
    var workspace = Workspace.CreateNew();
    var log = await UploadDataNodeStateToWorkspaceAsync(dataSet, workspace);
    await CommitToDatabase<DataNodeState,PartitionByReportingNode>(workspace, (Guid)workspace.Partition.GetCurrent(nameof(PartitionByReportingNode)), snapshot: false);    
    return log;
});

# Data Node Parameters

In [0]:
public async Task<ActivityLog> UploadDataNodeParameterToWorkspaceAsync(IDataSet dataSet, Guid targetPartitionByReportingNodeId, IWorkspace workspace)
{
    workspace.Reset(x => x.ResetInitializationRules().ResetCurrentPartitions());
    workspace.Initialize(x => x.FromSource(DataSource)
                               .DisableInitialization<RawVariable>()
                               .DisableInitialization<IfrsVariable>()
                               .DisableInitialization<DataNodeParameter>());
    Activity.Start();
    var args = await GetArgsFromMainAsync<PartitionByReportingNodeAndPeriod>(dataSet) with {ImportFormat = ImportFormats.DataNodeParameter};
    if(Activity.HasErrors()) return Activity.Finish();

    var storage = new ParsingStorage(args, DataSource, workspace);
    await storage.InitializeAsync();
    if(Activity.HasErrors()) return Activity.Finish();

    var singleDataNode = new List<string>();
    var interDataNode = new List<(string,string)>();
    
    var importLog = await Import.FromDataSet(dataSet)
                                .WithType<SingleDataNodeParameter>( (dataset, datarow) => {

                                    //read and validate DataNodes
                                    var dataNode = datarow.Field<string>(nameof(DataNode));
                                    if(!storage.IsValidDataNode(dataNode)) { ApplicationMessage.Log(Error.InvalidDataNode, dataNode); return null; }

                                    //check for duplicates
                                    if(singleDataNode.Contains(dataNode)) { ApplicationMessage.Log(Error.DuplicateSingleDataNode, dataNode); return null; }
                                    singleDataNode.Add(dataNode);
                                   
                                    //Instantiate SingleDataNodeParameter
                                    return new SingleDataNodeParameter {
                                        Year = args.Year,
                                        Month = args.Month,
                                        Partition = storage.TargetPartitionByReportingNode.Id,

                                        DataNode = dataNode,
                                        PremiumAllocation = (datarow.Field<object>(nameof(SingleDataNodeParameter.PremiumAllocation)))
                                                                .ToString().CheckStringForExponentialAndConvertToDouble(),
                                    };
                                })
                                .WithType<InterDataNodeParameter>( (dataset, datarow) => {

                                    //read and validate DataNodes
                                    var dataNode = datarow.Field<string>(nameof(InterDataNodeParameter.DataNode));
                                    if(!storage.IsValidDataNode(dataNode)) { ApplicationMessage.Log(Error.InvalidDataNode, dataNode); return null; }

                                    var linkedDataNode = datarow.Field<string>(nameof(InterDataNodeParameter.LinkedDataNode));
                                    if(!storage.IsValidDataNode(linkedDataNode)) { ApplicationMessage.Log(Error.InvalidDataNode, linkedDataNode); return null; }
                                    var dataNodes = new string[]{dataNode, linkedDataNode}.OrderBy(x => x).ToArray();

                                    //validate ReinsuranceGross Link
                                    var isDn1Reinsurance = storage.IsDataNodeReinsurance(dataNodes[0]);
                                    var isDn2Reinsurance = storage.IsDataNodeReinsurance(dataNodes[1]);
                                    var isGrossReinsuranceLink = (isDn1Reinsurance && !isDn2Reinsurance) != (!isDn1Reinsurance && isDn2Reinsurance);
                                    var reinsCov = (datarow.Field<object>(nameof(InterDataNodeParameter.ReinsuranceCoverage)))
                                                        .ToString().CheckStringForExponentialAndConvertToDouble();
                                    if(!isGrossReinsuranceLink && Math.Abs(reinsCov) > Precision )
                                        ApplicationMessage.Log(Error.ReinsuranceCoverageDataNode, dataNodes[0], dataNodes[1]);  // TODO: is this error or warning?

                                    //check for duplicates
                                    if(interDataNode.Contains((dataNodes[0], dataNodes[1])) || interDataNode.Contains((dataNodes[1], dataNodes[0])))
                                        ApplicationMessage.Log(Error.DuplicateInterDataNode, dataNodes[0], dataNodes[1]);  // TODO: is this error or warning?

                                    interDataNode.Add((dataNodes[0], dataNodes[1])); 
                                    //Instantiate InterDataNodeParameter
                                    return new InterDataNodeParameter {
                                        Year = args.Year,
                                        Month = args.Month,
                                        Partition = storage.TargetPartitionByReportingNode.Id,
                                        DataNode = dataNodes[0],
                                        LinkedDataNode = dataNodes[1],
                                        ReinsuranceCoverage = reinsCov,
                                    };
                                })
                                .WithTarget(workspace)
                                .ExecuteAsync();
    
    targetPartitionByReportingNodeId = storage.TargetPartitionByReportingNode.Id;
    return Activity.Finish().Merge(importLog);
}

In [0]:
Import.DefineFormat(ImportFormats.DataNodeParameter, async (options, dataSet) => {
    Guid partitionId = new Guid();
    var workspace = Workspace.CreateNew();
    var log = await UploadDataNodeParameterToWorkspaceAsync(dataSet, partitionId, workspace);
    await CommitToDatabase<SingleDataNodeParameter,PartitionByReportingNode>(workspace, partitionId, snapshot: false);
    await CommitToDatabase<InterDataNodeParameter,PartitionByReportingNode>(workspace, partitionId, snapshot: false);    
    
    return log;
});

# Cashflows

Variables are created upon import of Cash flow file.
Cash flows are firstly mapped into [RawVariables](../DataModel/DataStructure#raw-variables). These are then used as input for the calculation of the IFRS 17 business logic which computes [IfrsVariables](../DataModel/DataStructure#ifrs-variable).

Some computed variables depend on both cash flow and actual input, requiring recalculation at each new import.
In order to improve performance and maximize computational efficiency, we only (re)compute the set of variables that are expected to change given the underlying business logic.

In [0]:
public async Task<ActivityLog> ParseCashflowsToWorkspaceAsync(IDataSet dataSet, ImportArgs args, IWorkspace workspace)
{
    workspace.Reset(x => x.ResetInitializationRules().ResetCurrentPartitions());
    workspace.Initialize(x => x.FromSource(DataSource).DisableInitialization<RawVariable>().DisableInitialization<IfrsVariable>());
    
    Activity.Start();
    var parsingStorage = new ParsingStorage(args, DataSource, workspace);
    await parsingStorage.InitializeAsync();
    if(Activity.HasErrors()) return Activity.Finish();
    
    var hasAccidentYearColumn = dataSet.Tables[ImportFormats.Cashflow].Columns.Where(x => x.ColumnName == nameof(RawVariable.AccidentYear)).Count() > 0; 
    
    var importLog = await Import.FromDataSet(dataSet)
        .WithType<RawVariable> ( (dataset, datarow) => {
            var aocType = datarow.Field<string>(nameof(RawVariable.AocType));
            var novelty = datarow.Field<string>(nameof(RawVariable.Novelty));
            var dataNode = datarow.Field<string>(nameof(DataNode));
            
            if(!parsingStorage.DataNodeDataBySystemName.TryGetValue(dataNode, out var dataNodeData)) {
                ApplicationMessage.Log(Error.InvalidDataNode, dataNode);
                return null;
            }
            
            // Error if AocType is not present in the mapping
            if(!parsingStorage.AocTypeMap.Contains(new AocStep(aocType, novelty))) {
                ApplicationMessage.Log(Error.AocTypeMapNotFound, aocType, novelty);
                return null;
            }
                        
            // Filter out cash flows for DataNode that were created in the past and are still active and come with AocType = BOPI
            if(dataNodeData.Year < args.Year && aocType == AocTypes.BOP && novelty == Novelties.I) {
                ApplicationMessage.Log(Warning.ActiveDataNodeWithCashflowBOPI, dataNode);
                return null;
            }
                        
            var amountTypeFromFile = datarow.Field<string>(nameof(RawVariable.AmountType));
            var isEstimateType = parsingStorage.EstimateType.ContainsKey(amountTypeFromFile);
            var amountType = isEstimateType ? null : amountTypeFromFile;
            var estimateType = isEstimateType ? amountTypeFromFile : EstimateTypes.BE;
            
            var values = datarow.Table.Columns.Where(c => c.ColumnName.StartsWith(nameof(RawVariable.Values))).OrderBy(c => c.ColumnName.Length).ThenBy(c => c.ColumnName)
                                .Select(x => datarow.Field<string>(x.ColumnName).CheckStringForExponentialAndConvertToDouble()).ToArray();
            
            // Filter out empty raw variables for AocType != CL
            //TODO: extend this check for all mandatory step and not just for CL
            if(args.Scenario == null) {
                values = values.Prune();
                if(values.Length == 0 && aocType != AocTypes.CL) return null;
            }
                
            var item = new RawVariable {
                DataNode = dataNode,
                AocType = aocType,
                Novelty = novelty,
                AmountType = amountType,
                EstimateType = estimateType,
                AccidentYear = hasAccidentYearColumn && Int32.TryParse((datarow.Field<string>(nameof(RawVariable.AccidentYear))), out var accidentYear)
                                ? accidentYear
                                : (int?)null,
                Partition = parsingStorage.TargetPartitionByReportingNodeAndPeriod.Id,
                Values = Multiply(GetSign((aocType, amountType, estimateType, dataNodeData.IsReinsurance), parsingStorage.HierarchyCache), values)
            };
            return item;
        }, ImportFormats.Cashflow
    ).WithTarget(workspace).ExecuteAsync();
    
    await ValidateForDataNodeStateActiveAsync<RawVariable>(workspace, parsingStorage.DataNodeDataBySystemName);
    return Activity.Finish().Merge(importLog);
}

In [0]:
Import.DefineFormat(ImportFormats.Cashflow, async (options, dataSet) => {
    Activity.Start();
    var mainArgs = (await GetArgsFromMainAsync<PartitionByReportingNodeAndPeriod>(dataSet)) with {ImportFormat = ImportFormats.Cashflow};
    if(Activity.HasErrors()) return Activity.Finish();

    var targetArgs = await GetTargetArgsAsync(mainArgs);
    await DataNodeFactoryAsync(dataSet, ImportFormats.Cashflow, mainArgs);
    if(Activity.HasErrors()) return Activity.Finish();
    
    var workspace = Workspace.CreateNew();
    var log = await ParseCashflowsToWorkspaceAsync(dataSet, mainArgs, workspace);
    if(log.Errors.Any()) return Activity.Finish().Merge(log);

    foreach (var args in targetArgs) {
        log = log.Merge(await CalculateAndUploadVariablesAsync(args, workspace, args == mainArgs));
        if(log.Errors.Any()) return Activity.Finish().Merge(log);
    }
    return Activity.Finish().Merge(log); 
});

# Actuals

Variables are created upon import of Actual file.
Actuals are directly mapped into [IfrsVariables](../DataModel/DataStructure#ifrs-variable).



Some computed variables depend on both cash flow and actual input, requiring recalculation at each new import.
In order to improve performance and maximize computational efficiency, we only (re)compute the set of variables that are expected to change given the underlying business logic.

In [0]:
public async Task<ActivityLog> ParseActualsToWorkspaceAsync(IDataSet dataSet, ImportArgs args, IWorkspace workspace)
{
    workspace.Reset(x => x.ResetInitializationRules().ResetCurrentPartitions());
    workspace.Initialize(x => x.FromSource(DataSource)
                               .DisableInitialization<RawVariable>()
                               .DisableInitialization<IfrsVariable>());
    
    Activity.Start();
    var parsingStorage = new ParsingStorage(args, DataSource, workspace);
    await parsingStorage.InitializeAsync();
    if(Activity.HasErrors()) return Activity.Finish();

    var importLog = await Import.FromDataSet(dataSet)
        .WithType<IfrsVariable> ( (dataset, datarow) => {
            var dataNode = datarow.Field<string>(nameof(DataNode));
            if(!parsingStorage.DataNodeDataBySystemName.TryGetValue(dataNode, out var dataNodeData)) {
                ApplicationMessage.Log(Error.InvalidDataNode, dataNode);
                return null;
            }
            
            var valueType = datarow.Field<string>(ValueType);
            if(valueType == null) { 
                ApplicationMessage.Log(Error.ValueTypeNotFound); 
                return null; 
            }
            
            var amountType = parsingStorage.DimensionsWithExternalId[typeof(AmountType)].TryGetValue(valueType, out var at)? at : null;
            var isStdActual = parsingStorage.AmountType.ContainsKey(valueType);
            var estimateType = isStdActual? EstimateTypes.A : parsingStorage.DimensionsWithExternalId[typeof(EstimateType)].TryGetValue(valueType, out var et)? et : null;
            if(estimateType == null || amountType == null) { 
                ApplicationMessage.Log(Error.ValueTypeNotValid, valueType);
                return null;
            }
            
            var aocType = datarow.Field<string>(nameof(IfrsVariable.AocType));
            if((!isStdActual && aocType != AocTypes.CF && aocType != AocTypes.WO) || (isStdActual && aocType != AocTypes.CF) ) {
                ApplicationMessage.Log(Error.AocTypeNotValid, aocType);
                return null;
            }
                      
            var item = new IfrsVariable {
                DataNode = dataNode,
                AocType = aocType,
                Novelty = Novelties.C,
                AccidentYear = Int32.TryParse((datarow.Field<string>(nameof(IfrsVariable.AccidentYear))), out var tempAccYear)? tempAccYear : (int?)null,
                AmountType = amountType,
                EstimateType = estimateType,
                Partition = parsingStorage.TargetPartitionByReportingNodeAndPeriod.Id,
                Value = GetSign((aocType, amountType, estimateType, dataNodeData.IsReinsurance), parsingStorage.HierarchyCache) * datarow.Field<string>(nameof(IfrsVariable.Value)).CheckStringForExponentialAndConvertToDouble()
            };
            return item;
        }, ImportFormats.Actual
    ).WithTarget(workspace).ExecuteAsync();
    
    await ValidateForDataNodeStateActiveAsync<IfrsVariable>(workspace, parsingStorage.DataNodeDataBySystemName);
    return Activity.Finish().Merge(importLog);
}

In [0]:
Import.DefineFormat(ImportFormats.Actual, async (options, dataSet) => {
    Activity.Start();
    var mainArgs = (await GetArgsFromMainAsync<PartitionByReportingNodeAndPeriod>(dataSet)) with {ImportFormat = ImportFormats.Actual};
    if(Activity.HasErrors()) return Activity.Finish();

    var targetArgs = await GetTargetArgsAsync(mainArgs);
    await DataNodeFactoryAsync(dataSet, ImportFormats.Actual, mainArgs);
    if(Activity.HasErrors()) return Activity.Finish();

    var workspace = Workspace.CreateNew();
    var log = await ParseActualsToWorkspaceAsync(dataSet, mainArgs, workspace);
    if(log.Errors.Any()) return Activity.Finish().Merge(log);

    foreach (var args in targetArgs) {
        log = log.Merge(await CalculateAndUploadVariablesAsync(args, workspace, false)); 
        if(log.Errors.Any()) return Activity.Finish().Merge(log);
    }
    return Activity.Finish().Merge(log);
});

# Simple Value

Simple Value format imports [IfrsVariables](../DataModel/DataStructure#ifrs-variable) computed by an independent tool.
In this case our IFRS 17 calculation is not applied and variables are stored in the Database for being consumed in reports with our powerful reporting tooling.

In [0]:
public async Task<ActivityLog> ParseSimpleValueToWorkspaceAsync(IDataSet dataSet, ImportArgs args, Guid targetPartitionByReportingNodeAndPeriodId, IWorkspace workspace)
{
    workspace.Reset(x => x.ResetInitializationRules().ResetCurrentPartitions());
    workspace.Initialize(x => x.FromSource(DataSource)
                               .DisableInitialization<RawVariable>()
                               .DisableInitialization<IfrsVariable>());
    
    Activity.Start();
    var importFormat = args.ImportFormat;
    var parsingStorage = new ParsingStorage(args, DataSource, workspace);
    await parsingStorage.InitializeAsync();
    if(Activity.HasErrors()) return Activity.Finish(); 

    var importLog = await Import.FromDataSet(dataSet)
        .WithType<IfrsVariable> ( (dataset, datarow) => {
            var dataNode = parsingStorage.ValidateDataNode(datarow.Field<string>(nameof(DataNode)));
            var amountType = parsingStorage.ValidateAmountType(datarow.Field<string>(nameof(IfrsVariable.AmountType)));
            var estimateType = parsingStorage.ValidateEstimateType(datarow.Field<string>(nameof(IfrsVariable.EstimateType)), dataNode); //TODO LIC/LRC dependence
            var aocStep = importFormat == ImportFormats.SimpleValue 
                                    ? parsingStorage.ValidateAocStep(new AocStep (datarow.Field<string>(nameof(IfrsVariable.AocType)), 
                                                                                  datarow.Field<string>(nameof(IfrsVariable.Novelty))))
                                    : new AocStep(AocTypes.BOP, Novelties.I);
            var economicBasis = importFormat == ImportFormats.SimpleValue 
                                    ? datarow.Field<string>(nameof(IfrsVariable.EconomicBasis)) 
                                    : null;
            
            parsingStorage.ValidateEstimateTypeAndAmountType(estimateType, amountType);
            
            var iv = new IfrsVariable {
                DataNode = dataNode,
                AocType = aocStep.AocType,
                Novelty = aocStep.Novelty,
                AccidentYear = Int32.TryParse((datarow.Field<string>(nameof(IfrsVariable.AccidentYear))), out var accidentYear) ? accidentYear : (int?)null,
                AmountType = amountType,
                EstimateType = estimateType,
                EconomicBasis = economicBasis,
                Partition = parsingStorage.TargetPartitionByReportingNodeAndPeriod.Id,
                Value = GetSign((aocStep.AocType, amountType, estimateType, parsingStorage.IsDataNodeReinsurance(dataNode)), parsingStorage.HierarchyCache) * datarow.Field<string>(nameof(IfrsVariable.Value)).CheckStringForExponentialAndConvertToDouble()
            };
            return iv;
        }, importFormat // This should indicate the table name, not the input format
    ).WithTarget(workspace).ExecuteAsync();
    
    // Checking if there are inconsistencies in the TechnicalMarginEstimateTypes --> double entries in the steps where we expect to have unique values
    var invalidVariables = await workspace.Query<IfrsVariable>()
                            .Where(iv => parsingStorage.TechnicalMarginEstimateTypes.Contains(iv.EstimateType))
                            .Where(iv => iv.AocType == AocTypes.BOP || iv.AocType == AocTypes.EOP || iv.AocType == AocTypes.AM || iv.AocType == AocTypes.EA)
                            .GroupBy(iv => new {iv.DataNode, iv.AocType, iv.Novelty})
                            .Where(g => g.Count() > 1)
                            .Select(g => g.Key)
                            .ToArrayAsync();
    
    foreach (var iv in invalidVariables)
        ApplicationMessage.Log(Error.MultipleTechnicalMarginOpening, $"{iv.DataNode},{iv.AocType},{iv.Novelty}");
    
    await ValidateForDataNodeStateActiveAsync<IfrsVariable>(workspace, parsingStorage.DataNodeDataBySystemName);
    targetPartitionByReportingNodeAndPeriodId = parsingStorage.TargetPartitionByReportingNodeAndPeriod.Id;
    return Activity.Finish().Merge(importLog);
}

In [0]:
Import.DefineFormat(ImportFormats.SimpleValue, async (options, dataSet) => {
    Activity.Start();
    var args = (await GetArgsFromMainAsync<PartitionByReportingNodeAndPeriod>(dataSet)) with {ImportFormat = ImportFormats.SimpleValue};
    await DataNodeFactoryAsync(dataSet, ImportFormats.SimpleValue, args);
    if(Activity.HasErrors()) return Activity.Finish();
    
    Guid partitionId = new Guid();
    var workspace = Workspace.CreateNew();
    var parsingLog = await ParseSimpleValueToWorkspaceAsync(dataSet, args, partitionId, workspace);
    if(parsingLog.Errors.Any()) return Activity.Finish().Merge(parsingLog);
    
    workspace.Query<IfrsVariable>().Select(v => new {v.DataNode, v.AccidentYear}).Distinct();
    
    await CommitToDatabase<IfrsVariable,PartitionByReportingNodeAndPeriod>(workspace, partitionId, snapshot : true,
                                                                           filter : x => workspace.Query<IfrsVariable>().Select(v => v.DataNode).Distinct().Contains(x.DataNode));

    return Activity.Finish().Merge(parsingLog);
});

# Opening

Opening format imports [IfrsVariables](../DataModel/DataStructure#ifrs-variable) for the first period transition. It allows to import in force opening values (AoC Step BOP and novelty I) for EstimateTypes C, L, LR, AA, OA, DA.

In [0]:
Import.DefineFormat(ImportFormats.Opening, async (options, dataSet) => {
    Activity.Start();
    var args = (await GetArgsFromMainAsync<PartitionByReportingNodeAndPeriod>(dataSet)) with {ImportFormat = ImportFormats.Opening};
    await DataNodeFactoryAsync(dataSet, ImportFormats.Opening, args);
    if(Activity.HasErrors()) return Activity.Finish();

    Guid partitionId = new Guid();
    var workspace = Workspace.CreateNew();
    var parsingLog = await ParseSimpleValueToWorkspaceAsync(dataSet, args, partitionId, workspace);
    if(parsingLog.Errors.Any()) return Activity.Finish().Merge(parsingLog);

    var calculationLog = await CalculateAndUploadVariablesAsync(args, workspace, false); 
    if(calculationLog.Errors.Any()) return Activity.Finish().Merge(calculationLog);
    return Activity.Finish().Merge(parsingLog).Merge(calculationLog);
});